# Setting up the environment and importing libraries

In [3]:
import os
from dotenv import load_dotenv
load_dotenv('/home/parikh92/sciborg/ai/agents/.env')

openai_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_key

In [4]:
import os
import sys

sys.path.insert(1, "/home/parikh92/") 

from langchain_openai import ChatOpenAI
from sciborg.ai.agents.core import create_linqx_chat_agent
from sciborg.ai.chains.microservice import module_to_microservice
from sciborg.ai.chains.workflow import create_workflow_planner_chain, create_workflow_constructor_chain

from sciborg.testing.models.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from sciborg.core.library.base import BaseDriverMicroservice

In [5]:
file_path = '/home/parikh92/sciborg/ai/agents/driver_pubchem.json'

pubchem_command_microservice = module_to_microservice(PubChemCaller)
print(pubchem_command_microservice)

with open(file_path, 'w') as outfile:
    outfile.write(pubchem_command_microservice.model_dump_json(indent=2))

pubchem_driver_based_agent = create_linqx_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    verbose=True
)

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: get_assay_description\n    Microservice: PubChemCaller\n    UUID: 4d94c70c-d334-4a24-8b85-d527a5fc03df\n    Function Signature: (aid: int) -> dict\n    Docstring: Function purpose\nGet Assay description, protocol and comment on the scores for a given assay id.\n\nInputs\naid: assay ID\n\nreturns\ndict: a dictionary of description, protocol and comment for the given assay ID\n    ', 'text': {'name': 'get_assay_description', 'microservice': 'PubChemCaller', 'desc': 'A short description of the command', 'uuid': '4d94c70c-d334-4a24-8b85-d527a5fc03df', 'parameters': {'aid': {'name': 'aid', 'data_type': 'int', 'is_optional': False, 'is_list': False, 'desc': 'A short description of the parameter'}}, 'has_return': True, 're

In [7]:
file_path = '/home/parikh92/sciborg/ai/agents/driver_MicroSynth.json'

driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
print(type(driver_command_microservice))

with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: allocate_session\n    Microservice: MicrowaveSynthesizer\n    UUID: 9bf0a75e-8378-4f2e-aec1-a827f0f19135\n    Function Signature: () -> dict\n    Docstring: Allocates a session on the microwave synthesizer.\nMust be called prior to any other action.\n\nreturns\nsession_ID the id of the allocated session\n    ', 'text': {'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '9bf0a75e-8378-4f2e-aec1-a827f0f19135', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}}
{'query': '\n    Create a command from the below information. \n    Do not

# PubChem Agent

Since pubchem agent requires usage of multiple tools (generally) in a specific sequence, we will be opting for path based benchmarking here.

In [ ]:
from sciborg.utils.benchmarking.path import AgentPathBenchmarker
from pydantic import BaseModel
from typing import Literal
from langchain_openai import ChatOpenAI

class CidFromNameSchema(BaseModel):
    inp: Literal['pzm21', 'PZM21'] 

class AssayResultsFromCompound(BaseModel):
    cid: Literal['121596705', '124186394']
    activity_name: Literal['Ki']

class AssayDescriptionFromAid(BaseModel):
    aid: Literal['1806958', '1609954', '1573390'] 


#need to define paths here
path = [
    [
        ('get_cid_from_name', CidFromNameSchema),
        ('get_assay_results_for_compund',AssayResultsFromCompound),
        ('get_assay_description',AssayDescriptionFromAid)
    ],
    [
        ('get_cid_from_name', CidFromNameSchema),
        ('get_assay_results_for_compund',AssayResultsFromCompound),
        ('get_assay_results_for_compund',AssayResultsFromCompound),
        ('get_assay_description',AssayDescriptionFromAid)
    ]
]

executor_kwargs = {
    'microservice': pubchem_command_microservice,
    'llm': ChatOpenAI(model='gpt-4'),
    'human_interaction': False,
    'agent_as_a_tool': None,
    'use_linqx_tools': True, #this will use the tools from the driver which is pubchem
}

output_benchmark = AgentPathBenchmarker(
    executor_fn=create_linqx_chat_agent,
    executor_kwargs=executor_kwargs,
    initial_input="What is the Ki value of the compound PZM21 when it binds to the Mu opioid receptor? Please select one specific assay used to determine this value, provide a citation for that particular assay, and give a detailed description of how the assay was conducted.",
    desired_output=path,
    verbose=True,
    notebook=True
)

In [6]:
output_benchmark.benchmark(20)
#be very careful with the question, as a variation in question can lead to a different path

  0%|          | 0/20 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: First, I need to get the Compound ID (CID) for PZM21. I'll use the get_cid_from_name function for this.

Action:
```
{
    "action": "get_cid_from_name",
    "action_input": {
        "inp": "PZM21"
    }
}
```{'response': {'', '121596705', '124186394'}}The compound ID for PZM21 is '121596705' or '124186394'. Now I'll use the get_assay_results_for_compund function to get the Ki value when PZM21 binds to the Mu opioid receptor. I'll filter the results to only include assays that measure the Ki value.

Action:
```
{
    "action": "get_assay_results_for_compund",
    "action_input": {
        "cid": "121596705",
        "activity_name": "Ki"
    }
}
```{'AID': {1: '1573390', 5: '1609954', 17: '1806958', 18: '1806958', 19: '1806958', 20: '1806958'}, 'Panel Member ID': {1: '', 5: '', 17: '', 18: '', 19: '', 20: ''}, 'SID': {1: '440143506', 5: '440143506', 17: '440726618', 18: '440726618', 19: '440726618', 20: '440726618'}, 'CID': {1: '1215967

In [9]:
output_benchmark.benchmark(20)

  0%|          | 0/20 [00:00<?, ?it/s]

https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1573390/description/JSON
- Success on output key: intermediate_steps, matched a desired output
- Agent action path:
get_cid_from_name: {'inp': 'PZM21'}
get_assay_results_for_compund: {'cid': '121596705', 'activity_name': 'Ki'}
get_assay_description: {'aid': '1573390'}

- Iteration 1: Success
- Success: 1, Fail: 0, Total: 1
- Benchmarking Score: 1.0
--------------------
https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1573390/description/JSON
- Success on output key: intermediate_steps, matched a desired output
- Agent action path:
get_cid_from_name: {'inp': 'PZM21'}
get_assay_results_for_compund: {'cid': '121596705', 'activity_name': 'Ki'}
get_assay_description: {'aid': '1573390'}

- Iteration 2: Success
- Success: 2, Fail: 0, Total: 2
- Benchmarking Score: 1.0
--------------------
https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1573390/description/JSON
names Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown o